In [1]:
 import pandas as pd
import numpy as np
import datetime
import calendar
import math
import geopandas as gpd
import folium
import psutil
import random
import time
from os import listdir
from os.path import isfile, join

def READHDR(link):
    f = open(link, "r", encoding="utf-8")
    header = pd.DataFrame({'col1': f.read().splitlines()})
    f.close()
    header['value'] = header['col1'].apply(lambda x: x.split()[1])
    header['col1'] = header['col1'].apply(lambda x: x.split()[0])
    index = header.col1.unique()
    header.drop(columns='col1', inplace=True)
    header.set_index(index, inplace=True)
    return header

def DF_UTIL(header,city):
    ulxmap = pd.to_numeric(header.iloc[5])
    ulymap = pd.to_numeric(header.iloc[6])
    xdim = pd.to_numeric(header.iloc[7])
    ydim = pd.to_numeric(header.iloc[8])
    lonLatBbox = city.coordinates
    minj = math.floor((lonLatBbox[0] - ulxmap + xdim / 2) / xdim)
    maxi = math.ceil((ulymap - lonLatBbox[1] + ydim / 2) / ydim)

    maxj = math.ceil((lonLatBbox[2] - ulxmap + xdim / 2) / xdim)
    mini = math.floor((ulymap - lonLatBbox[3] + ydim / 2) / ydim)

    nrows = maxi - mini
    ncols = maxj - minj
    return pd.DataFrame(np.array([minj, mini, nrows,ncols]), columns=['valori'], index =['minj', 'mini','nrows', 'ncols'])

#Melt mette tutto in colonna e non più griglia
def BILREADER_CORRETTO(filename, header,cnt):
    ncols = pd.to_numeric(header.loc['NCOLS', 'value'])
    nrows = pd.to_numeric(header.loc['NROWS', 'value'])

    try:
        cellecentocinquanta = pd.DataFrame(columns=np.arange(ncols))
        for i in range(nrows):
             riga = pd.Series(np.fromfile(filename, dtype='<u2', count=ncols, sep="", offset=i * ncols * 2))
             cellecentocinquanta = cellecentocinquanta.append(riga, ignore_index=True)
        # cellecentocinquanta = cellecentocinquanta.head(50)
        # cellecentocinquanta
        cellecentocinquanta = cellecentocinquanta.reset_index().melt(id_vars= ['index'], value_name= str(cnt))
        cellecentocinquanta['riga-colonna'] = cellecentocinquanta['index'].astype(str) +'-'+cellecentocinquanta.variable.astype(str)
        cellecentocinquanta= cellecentocinquanta[['riga-colonna', str(cnt)]]
        return cellecentocinquanta
    except OSError as e:
        print(f'manca {cnt}.bil')
        df = pd.DataFrame(columns= np.arange(ncols), index= np.arange(nrows))
        df = df.reset_index().melt(id_vars= ['index'], value_name= str(cnt))
        df['riga-colonna'] = df['index'].astype(str) +'-'+df.variable.astype(str)
        df= df[['riga-colonna', str(cnt)]]
        return df



class City:
    def __init__(self, name, coordinates):
        self.name = name
        self.coordinates = coordinates


def data_amg(data):
    return data.strftime("%Y%m%d")


def data_amg_om(data):
    return data.strftime("%Y%m%d_%H%M")


#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#campiare link alla cartina all'SHP dell'Europa
def SEZIONI2COMUNI(header, cartella_dove_printare, mappa):
    europa = gpd.GeoDataFrame(gpd.read_file('D:/Università/Tirocinio/Analisi Dati/Grezzo Niki/SHP/Med Land/Med_land.shp')['geometry'])
    out = open(f'{cartella_dove_printare}/sezione2comuneDEF.csv', 'a')
    out.write(f',comune\n')
    comune = ''

    nrows =np.arange(int(header.iloc[3,0]))
    ncols = np.arange(int(header.iloc[4,0]))
    x = np.array(np.meshgrid(nrows,ncols )).transpose().reshape((int(header.iloc[3,0])*int(header.iloc[4,0]),2))
    inizio = time.time()
    for i in range(len(x)):
        if i % (int(header.iloc[4,0])*150) == 0:
            fine = time.time()
            print(f'checkpoint: {fine-inizio}')
            inizio = time.time()
        punto = TROVACOORDINATE( np.array2string(x[i],separator='-')[1:-1].replace(' ',''),yx,UL) #[y,x]
        comune = QUADRATINO2COMUNE(punto, mappa, comune, europa)

        out.write(f'{np.array2string(x[i],separator="-")[1:-1].replace(" ","")},{comune}\n')

    out.close()


def SEZIONI3COMUNI(header, cartella_dove_printare, mappa):
    europa = gpd.GeoDataFrame(
        gpd.read_file('D:/Università/Tirocinio/Analisi Dati/Grezzo Niki/SHP/Med Land/Med_land.shp')['geometry'])
    out = open(f'{cartella_dove_printare}/Camerino.csv', 'a')
    out.write(f',Sezione\n')
    sezione = ''

    nrows = np.arange(int(header.iloc[3, 0]))
    ncols = np.arange(int(header.iloc[4, 0]))
    x = np.array(np.meshgrid(nrows, ncols)).transpose().reshape((int(header.iloc[3, 0]) * int(header.iloc[4, 0]), 2))
    inizio = time.time()
    for i in range(len(x)):
        if i % (int(header.iloc[4, 0]) * 150) == 0:
            fine = time.time()
            print(f'checkpoint: {fine - inizio}')
            inizio = time.time()
        punto = TROVACOORDINATE(np.array2string(x[i], separator='-')[1:-1].replace(' ', ''), yx, UL)  #[y,x]
        sezione = QUADRATINO3COMUNE(punto, mappa, sezione, europa)

        out.write(f'{np.array2string(x[i], separator="-")[1:-1].replace(" ", "")},{sezione}\n')

    out.close()


def QUADRATINO3COMUNE(punto, comuni, sezione, europa):
    if punto[1]> 12.96869 and punto[1]< 13.17837:
        if punto[0]< 43.20578 and punto [0]>43.06468:

            punto = gpd.points_from_xy([punto[1]], [punto[0]])[0]
            if sezione in comuni.SEZ.values:
                if comuni.loc[comuni.SEZ == sezione].iloc[0, 1].contains(punto):
                    return sezione

            if europa.iloc[0, 0].contains(punto):
                for j in range(len(comuni.geometry)):
                    if comuni.iloc[j, 1].contains(punto):
                        sezione = comuni.iloc[j, 0]

                        return sezione
            else:
                sezione = 'Non Camerino'
                return sezione
    else: return 'Non Camerino'

def TROVACOORDINATE(indici, yx, UL):
    rigcol = pd.to_numeric(pd.Series(indici.split("-")))
    delta = yx * rigcol
    coordinate = [UL[0] - delta[0] - yx[0]/2, UL[1] + delta[1] + yx[1]/2]
    return coordinate

def QUADRATINO2COMUNE(punto, comuni, comu, europa):
    if punto[1] == 11.835000385299999:
        # valore di x del primo elemento di ogni riga
        comu = ''
    if comu == 'Mare':

        return comu
    punto = gpd.points_from_xy([punto[1]], [punto[0]])[0]
    if comu in comuni.COMUNE.values:
        if comuni.loc[comuni.COMUNE == comu].iloc[0,1].contains(punto):
            return comu



    if europa.iloc[0,0].contains(punto):
        for j in range(len(comuni.geometry)):
            if comuni.iloc[j,1].contains(punto):
                comu = comuni.iloc[j,0]

                return comu
    else:
        comu = 'Mare'
        return comu

def PUNTISEZIONE(sezionetocomune, cartella_dove_printare):
    sezionetocomune.reset_index(inplace= True)
    sezionetocomune.rename( columns={'index':'sezione'}, inplace=True )
    sezionetocomune['coordinate'] = sezionetocomune.sezione.apply(lambda c: TROVACOORDINATE(c,yx,UL))
    sezionetocomune['Longitudine'] = sezionetocomune.coordinate.apply(lambda l: l[1])
    sezionetocomune.coordinate = sezionetocomune.coordinate.apply(lambda u: u[0])
    sezionetocomune.rename(columns = {'coordinate' : 'Latitudine'}, inplace= True)

    congeometry = gpd.GeoDataFrame(sezionetocomune, geometry=gpd.points_from_xy(sezionetocomune.Longitudine, sezionetocomune.Latitudine))
    congeometry = congeometry[['sezione', 'comune', 'geometry']]
    congeometry.to_file(f'{cartella_dove_printare}/sezione2comuniSHP.shp')
    return congeometry



#Mask
def TOCSVGROUPBY(hb, cartella_dove_printare, city, cal, end, sezionetocomune):

    hb.set_index(hb['riga-colonna'].unique(),inplace = True)
    hb = hb.drop(columns = 'riga-colonna')
    for i in hb.columns:
        hb[i].mask((hb[i]== 65535), inplace = True)
    hb['value'] = hb.mean(axis = 1)

    #hb = hb['value'].reset_index()
    # sezionetocomune.reset_index(inplace = True)
    # hb.rename(columns = {'index': 'sezione'}, inplace = True)
    # sezionetocomune.rename(columns = {'index': 'sezione'}, inplace = True)
    # hb = pd.DataFrame(hb).join(pd.DataFrame(sezionetocomune))# tiene tutte le sezioni passando comune nan dove non è definito
    hb = pd.DataFrame(hb).merge(pd.DataFrame(sezionetocomune),left_index=True, right_index=True) #tiene solo le righe che hanno comune
    hb.comune.fillna('Mare', inplace = True)
    hb = hb.groupby('comune').aggregate('value').sum().reset_index()
    hb['time'] = data_amg_om(cal)
    hb.to_csv(f'{cartella_dove_printare}/{city.name}_{data_amg(cal)}_{data_amg(end)}_24.csv', mode = 'a', index = False, header = None)


def run(city, start, end, header, sezionetocomune, cartella_dei_dati, cartella_dove_printare):
        print('inizio Run')
        delta = datetime.timedelta(hours= 1)
        cal = start
        # print(f'ram iniziale in mb: {psutil.virtual_memory().used/1000000}') ???????????

        while cal < end:
            print(f'processo {cal.date()}')
            data = data_amg(cal)#%Y%m%d
            giorno = f'{cartella_dei_dati}/{city.name}/{data}/'    #dove leggiamo
            bils = [x for x in [f for f in listdir(giorno) if isfile(join(giorno, f))] if x.endswith('bil')]
            cnt = 0
            while (cnt < 96) & (cal < end):
                # hb = pd.DataFrame(np.zeros((pd.to_numeric(header.loc['NROWS', 'value']), pd.to_numeric(header.loc['NCOLS', 'value']))))
                inizio = time.time()

                hb = BILREADER_CORRETTO(f'{giorno}{cnt}.bil', header,cnt)

                cnt += 1
                for f in range(3):
                    hb = hb.merge(BILREADER_CORRETTO(f'{giorno}{cnt}.bil', header,cnt))
                    cnt += 1

                TOCSVGROUPBY(hb, cartella_dove_printare, city, cal, end, sezionetocomune)

                print(f'Fatte le {cal.hour}. Per questo csv ho impiegato: {time.time() - inizio}')
                cal += delta


                # break





def MAPPASEZIONECOMUNI (sezioni, mappa, cartella_dove_printare, densità_puntini):
    sezioni = sezioni.rename(columns = {'comune' : 'COMUNE'})
    m = folium.Map(location=[mappa.centroid.y.mean(), mappa.centroid.x.mean()], tiles="cartodbpositron", zoom_start=8)
    #  = sezioni.head(8000)
    comuni = {}
    for i in range(len(sezioni)):
        if i % densità_puntini == 0 :
            if i % 10000 == 0: print(f'siamo arrivati: {i}')
            if sezioni.iloc[i,1] in comuni:
                folium.Circle(
                radius=75,
                location=[pd.to_numeric(sezioni.iloc[i,2])[1], pd.to_numeric(sezioni.iloc[i,2])[0]],
                color= comuni.get(sezioni.iloc[i,1]),
                fill=True,
                fill_color = comuni.get(sezioni.iloc[i,1]),
                fill_opacity = 1.0,
                ).add_to(m)
            else:
                comuni[sezioni.iloc[i,1]] = '#'+''.join(random.choice('ABCDEF1234567890') for k in range(6))
                folium.Circle(
                radius=75,
                location= [pd.to_numeric(sezioni.iloc[i,2])[1], pd.to_numeric(sezioni.iloc[i,2])[0]],
                color= comuni.get(sezioni.iloc[i,1]),
                fill=True,
                fill_color = comuni.get(sezioni.iloc[i,1]),
                fill_opacity = 1.0,
                ).add_to(m)
    # folium.LayerControl().add_to(mappa)
    return m

def CREACARTINA(percorso_file, cartella_dove_printare):
    map = gpd.read_file(percorso_file)
    # map.loc[map['COMUNE'] == 'Carpi'].plot()
    # map.plot()
    # map.loc[pd.to_numeric(map['COD_REG']) == 20].plot()
    map = map.loc[pd.to_numeric(map['COD_REG']) > 7]
    map = map.loc[pd.to_numeric(map['COD_REG']) < 14]
    map = map[['COMUNE','geometry']]
    return map
#DATI

area = "CentroItalia"
coordinates = [13.001457, 42.992867, 13.941032, 43.661072]
city = City(area, coordinates)
UL = [43.7672915262, 11.8341670519]  #Upper Left Y,X Map
yx = pd.Series([0.0012500001, 0.0016666668])  #Dimensione y,x sezione
month = '09' #mese
year = '2016' #anno
start = datetime.datetime(int(year), int(month), 8, 0, 0, 0)
end = datetime.datetime(int(year), int(month), 9, 0, 0, 0)






cartella_dei_dati = f'D:/Università/Tirocinio/Analisi Dati/Dati Grezzi'
cartella_dove_printare = f'D:/Università/Tirocinio/Analisi Dati/Grezzo Niki'

In [ ]:
header = READHDR(f'{cartella_dei_dati}/{area}/20150401/0.hdr')
df_util = DF_UTIL(header,city)
mappa = CREACARTINA('D:/Università/Tirocinio/Analisi Dati/Grezzo Niki/SHP/Comuni_2016/comuni2016.shp', cartella_dove_printare)
# minj =df_util.loc['minj','valori']
# mini =df_util.loc['mini','valori']
# nrows =df_util.loc['nrows','valori']
# ncols =df_util.loc['ncols','valori']





SEZIONI2COMUNI(header, cartella_dove_printare, mappa)
sezionetocomune = pd.read_csv(f'{cartella_dove_printare}/sezione2comuneDEF.csv', index_col= 0)
# PUNTISEZIONE(sezionetocomune, cartella_dove_printare)
# sezioni = gpd.read_file(f'{cartella_dove_printare}/sezione2comuneSHP.shp')
# run(city, start, end, header, sezionetocomune, cartella_dei_dati, cartella_dove_printare)

# map = MAPPASEZIONECOMUNI (sezioni, mappa, cartella_dove_printare, 7)
# map

In [3]:
Camerino = gpd.read_file('D:/Università/Tirocinio/Analisi Dati/Valutazioni/Camerino.shp')

In [26]:
Camerino= Camerino[['SEZ','geometry']]

In [4]:
header = READHDR(f'{cartella_dei_dati}/{area}/20150401/0.hdr')
SEZIONI3COMUNI(header, cartella_dove_printare, Camerino)



checkpoint: 0.0


KeyboardInterrupt: 